# Import

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import HBox, VBox
from datetime import date
from datetime import timedelta

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Données diverses

In [3]:
Solde_initial = 8048.07

Annee_dep = 2019

date_jour = date.today()

# Fonctions diverses

In [4]:
def reconstruct_list(mini, maxi):
    return [i for i in range(mini, maxi+1)]

# Création des DataFrame principaux

## Création du DataFrame global 

### Comptabilité personnelle

In [5]:
dossier = "G:\\Mon Drive\\Info fun\\Compta - pipenv\\Extract_for_python\\"

##### 2019

In [6]:
BDD_2019 = "extract_2019.xlsx"
df_2019 = pd.read_excel(dossier+BDD_2019, header=2)
df_2019.dropna(inplace=True, subset=["Date"])

##### 2020

In [7]:
BDD_2020 = "extract_2020.xlsx"
df_2020 = pd.read_excel(dossier+BDD_2020, header=2)
df_2020.dropna(inplace=True, subset=["Date"])

##### 2021

In [8]:
BDD_2021 = "extract_2021.xlsx"
df_2021 = pd.read_excel(dossier+BDD_2021, header=2)
df_2021.dropna(inplace=True, subset=["Date"])

##### 2022 - temp

In [9]:
BDD_2022 = "extract_20220101_20220904.xlsx"
df_2022 = pd.read_excel(dossier+BDD_2022, header=2)
df_2022.dropna(inplace=True, subset=["Date"])

##### Total

In [10]:
df = pd.concat([df_2019, df_2020, df_2021, df_2022], ignore_index=True)

In [11]:
#on crée les catégories
df_temp = df.Catégorie.str.split(pat=" > ", expand=True)

df["Catégorie_ppal"] = df_temp[0]
df["Catégorie_sec"] = df_temp.loc[:,1:len(df_temp.columns)]
df["Crédit"] = pd.to_numeric(df["Crédit"])

df.head()

,Date,Description,Catégorie,Mode de paiement,Tiers,Réf,Débit,Crédit,P,Catégorie_ppal,Catégorie_sec
0,2019-01-07,NaN,R04 - Virement | Chèque > 01 - Mireille Maisons,Non indiqué,Aucun,NaN,NaN,150.0,1.0,R04 - Virement | Chèque,01 - Mireille Maisons
1,2019-01-07,NaN,R04 - Virement | Chèque > xx - Carolane Dupont,Non indiqué,Aucun,NaN,NaN,325.0,1.0,R04 - Virement | Chèque,xx - Carolane Dupont
2,2019-01-07,NaN,D11 - Transport > 01 - Ilevia,Non indiqué,Aucun,NaN,14.45,NaN,1.0,D11 - Transport,01 - Ilevia
3,2019-01-07,NaN,D03 - Restaurant > 03 - KFC,Non indiqué,Aucun,NaN,29.00,NaN,1.0,D03 - Restaurant,03 - KFC
4,2019-01-07,NaN,R04 - Virement | Chèque > 04 - Gislaine Maisons,Non indiqué,Aucun,NaN,NaN,200.0,1.0,R04 - Virement | Chèque,04 - Gislaine Maisons


## Création du DataFrame Dépense 

In [12]:
df_dépense = df[(df["Mode de paiement"]!="Virement interne") & (df["Crédit"].isna())]
df_dépense.head()

,Date,Description,Catégorie,Mode de paiement,Tiers,Réf,Débit,Crédit,P,Catégorie_ppal,Catégorie_sec
2,2019-01-07,NaN,D11 - Transport > 01 - Ilevia,Non indiqué,Aucun,NaN,14.45,NaN,1.0,D11 - Transport,01 - Ilevia
3,2019-01-07,NaN,D03 - Restaurant > 03 - KFC,Non indiqué,Aucun,NaN,29.00,NaN,1.0,D03 - Restaurant,03 - KFC
5,2019-01-09,NaN,D06 - Internet | Téléphone > xx - RED | Tel. C...,Non indiqué,Aucun,NaN,10.00,NaN,1.0,D06 - Internet | Téléphone,xx - RED | Tel. Cléme
6,2019-01-13,NaN,D11 - Transport > 01 - Ilevia,Non indiqué,Aucun,NaN,14.45,NaN,1.0,D11 - Transport,01 - Ilevia
7,2019-01-14,NaN,D01 - Courses > 01 - Boulangerie,Non indiqué,Aucun,NaN,2.00,NaN,1.0,D01 - Courses,01 - Boulangerie


## Création du DataFrame Revenu 

In [13]:
df_revenu = df[(df["Mode de paiement"]!="Virement interne") & (df["Débit"].isna())]
df_revenu.head()

,Date,Description,Catégorie,Mode de paiement,Tiers,Réf,Débit,Crédit,P,Catégorie_ppal,Catégorie_sec
0,2019-01-07,NaN,R04 - Virement | Chèque > 01 - Mireille Maisons,Non indiqué,Aucun,NaN,NaN,150.00,1.0,R04 - Virement | Chèque,01 - Mireille Maisons
1,2019-01-07,NaN,R04 - Virement | Chèque > xx - Carolane Dupont,Non indiqué,Aucun,NaN,NaN,325.00,1.0,R04 - Virement | Chèque,xx - Carolane Dupont
4,2019-01-07,NaN,R04 - Virement | Chèque > 04 - Gislaine Maisons,Non indiqué,Aucun,NaN,NaN,200.00,1.0,R04 - Virement | Chèque,04 - Gislaine Maisons
26,2019-01-21,NaN,R02 - Remboursement > 01 - Amazon,Non indiqué,Aucun,NaN,NaN,54.99,1.0,R02 - Remboursement,01 - Amazon
36,2019-01-31,NaN,R02 - Remboursement > xx - Undiz,Non indiqué,Aucun,NaN,NaN,66.55,1.0,R02 - Remboursement,xx - Undiz


# Dashboard - Code

## Définition widget

In [14]:
option_year = sorted(list(df_dépense["Date"].dt.year.unique()))
option_cat_ppal_dep = sorted(list(df_dépense["Catégorie_ppal"].unique()))
option_cat_ppal_rev = sorted(list(df_revenu["Catégorie_ppal"].unique()))

pick_year = widgets.SelectMultiple(options=option_year,
                                   description='Pick a year :',
                                   value=[date_jour.year],
                                   layout={'width': 'max-content'},
                                   disabled=False)

pick_year = widgets.IntRangeSlider(value=[max(option_year), max(option_year)],
                                   min=min(option_year),
                                   max=max(option_year),
                                   step=1,
                                   description='Pick a year :',
                                   disabled=False,
                                   continuous_update=False,
                                   orientation='horizontal',
                                   readout=True,
                                   readout_format='d')

pick_cat_ppal_dep = widgets.SelectMultiple(options=option_cat_ppal_dep,
                                       description='Pick a cat. :',
                                       value=option_cat_ppal_dep,
                                       layout={'width': 'max-content'},
                                       disabled=False,
                                       rows=15)

pick_cat_ppal_rev = widgets.SelectMultiple(options=option_cat_ppal_rev,
                                       description='Pick a cat. :',
                                       value=option_cat_ppal_rev,
                                       layout={'width': 'max-content'},
                                       disabled=False,
                                       rows=15)

pick_nb_month = widgets.IntSlider(
    value=3,
    min=1,
    max=12,
    step=1,
    description='Nb de mois :',
    disabled=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display_multi_dep = HBox([pick_year, pick_cat_ppal_dep])
display_multi_rev = HBox([pick_year, pick_cat_ppal_rev])

## Soldes

In [15]:
def trace_solde():
    df_month = df[(df["Mode de paiement"]!="Virement interne")]

    df_month = df_month.groupby(pd.Grouper(key="Date", freq="1M")).sum().reset_index()
    df_month["Solde"] = df_month["Crédit"] - df_month["Débit"]
    df_month["Solde_tot"] = df_month["Solde"].cumsum() + Solde_initial
    
    df_month = df_month[(df_month["Date"].dt.year.isin(reconstruct_list(min(pick_year.value), max(pick_year.value))))]
    
    df_month["Date"] = df_month["Date"].apply(lambda x: x - timedelta(days=x.day - 1))

    fig = go.Figure(data=[go.Bar(name='Débit', x=df_month["Date"], y=df_month["Débit"], marker_color='red'),
                          go.Bar(name='Crédit', x=df_month["Date"], y=df_month["Crédit"], marker_color='green'),
                          go.Bar(name='Solde', x=df_month["Date"], y=df_month["Solde"], marker_color='orange'),
                          go.Scatter(name='Solde comptes', x=df_month["Date"], y=df_month["Solde_tot"], marker_color='blue')
    ])
    
    fig.update_layout(title=f"Soldes Mensuels {min(pick_year.value)}-{max(pick_year.value)}",
                      xaxis_title="Mois",
                      yaxis_title="Montant",
                      legend_title="Types de soldes",
                      barmode='group')

    fig.show()

## Solde moyen

In [16]:
def trace_solde_moy(nb_mois):
    df_month = df[(df["Mode de paiement"]!="Virement interne")]

    df_month = df_month.groupby(pd.Grouper(key="Date", freq="1M")).sum().reset_index()
    df_month["Solde"] = df_month["Crédit"] - df_month["Débit"]
    df_month["Solde_tot"] = df_month["Solde"].cumsum() + Solde_initial

    df_month = df_month[(df_month["Date"].dt.year.isin(reconstruct_list(min(pick_year.value), max(pick_year.value))))]

    df_month['Solde moyen'] = df_month['Solde'].rolling(nb_mois).mean()

    df_month["Date"] = df_month["Date"].apply(lambda x: x - timedelta(days=x.day - 1))
        
    # print(f'Moyenne des soldes sur la période : {df_month["Solde"].mean():.2f}€')

    fig = go.Figure(data=[go.Bar(name='Solde',
                                 x=df_month["Date"],
                                 y=df_month["Solde"],
                                 marker_color='orange'),
                          go.Scatter(name=f'Solde moyen {nb_mois} mois',
                                     x=df_month["Date"],
                                     y=df_month["Solde moyen"],
                                     marker_color='blue'),
                          go.Scatter(name='Solde moyen',
                                     x=df_month["Date"],
                                     y=[df_month["Solde"].mean()]*len(df_month["Date"]),
                                     mode='lines',
                                     line=dict(color='black',
                                               dash='dash',
                                               width=1))
        ])


    fig.update_layout(title=f"Solde Mensuel {min(pick_year.value)}-{max(pick_year.value)}",
                      xaxis_title="Mois",
                      yaxis_title="Montant",
                      legend_title="Types de soldes",
                      barmode='group')
    fig.show()

## Détails Dépenses

In [17]:
def det_dep():
    df_dépense_year_cat = df_dépense[(df_dépense["Date"].dt.year.isin(pick_year.value)) &
                                     (df_dépense["Catégorie_ppal"].isin(pick_cat_ppal_dep.value))]

    fig = px.sunburst(df_dépense_year_cat, path=['Catégorie_ppal'], values='Débit', title=f"Dépenses {min(pick_year.value)}-{max(pick_year.value)}")
    fig.show()

    fig = px.sunburst(df_dépense_year_cat, path=['Catégorie_ppal', 'Catégorie_sec'], values='Débit')
    fig.show()

## Détails Revenus

In [18]:
def det_rev():
    df_revenu_year_cat = df_revenu[(df_revenu["Date"].dt.year.isin(pick_year.value)) &
                                     (df_revenu["Catégorie_ppal"].isin(pick_cat_ppal_rev.value))]

    fig = px.sunburst(df_revenu_year_cat, path=['Catégorie_ppal'], values='Crédit', title=f"Revenus {min(pick_year.value)}-{max(pick_year.value)}")
    fig.show()

    fig = px.sunburst(df_revenu_year_cat, path=['Catégorie_ppal', 'Catégorie_sec'], values='Crédit')
    fig.show()

## Détails dépenses mensuels

In [19]:
def dep_mens():
    df_temp_year_cat = df_dépense[(df_dépense["Date"].dt.year.isin(pick_year.value)) &
                              (df_dépense["Catégorie_ppal"].isin(pick_cat_ppal_dep.value))]

    for idx, row in df.iterrows():
        df_temp_year_cat.loc[idx, "Date_mois"] = row["Date"] - timedelta(days=row["Date"].day-1)

    df_month = df_temp_year_cat[(df_temp_year_cat["Mode de paiement"]!="Virement interne")]

    df_month = df_month.groupby(by=["Date_mois", "Catégorie_ppal"]).sum().reset_index()

    fig = px.bar(df_month, x="Date_mois", y="Débit", color="Catégorie_ppal", title=f"Détails dépenses {min(pick_year.value)}-{max(pick_year.value)}")
    fig.show()

## Détails revenus mensuels

In [20]:
def rev_mens():
    df_temp_year_cat = df_revenu[(df_revenu["Date"].dt.year.isin(pick_year.value)) &
                                 (df_revenu["Catégorie_ppal"].isin(pick_cat_ppal_rev.value))]

    for idx, row in df.iterrows():
        df_temp_year_cat.loc[idx, "Date_mois"] = row["Date"] - timedelta(days=row["Date"].day-1)

    df_month = df_temp_year_cat[(df_temp_year_cat["Mode de paiement"]!="Virement interne")]

    df_month = df_month.groupby(by=["Date_mois", "Catégorie_ppal"]).sum().reset_index()

    fig = px.bar(df_month, x="Date_mois", y="Crédit", color="Catégorie_ppal", title=f"Détails revenus {min(pick_year.value)}-{max(pick_year.value)}")
    fig.show()

# Dashboard - Affichage

## Soldes

In [21]:
display(pick_year)
trace_solde()

IntRangeSlider(value=(2022, 2022), continuous_update=False, description='Pick a year :', max=2022, min=2019)

## Solde moyen

In [22]:
display(pick_year)
display(pick_nb_month)
trace_solde_moy(pick_nb_month.value) #nb de mois de moyenne

IntRangeSlider(value=(2022, 2022), continuous_update=False, description='Pick a year :', max=2022, min=2019)

IntSlider(value=3, description='Nb de mois :', max=12, min=1)

## Détails Dépenses 

In [23]:
display(display_multi_dep)
det_dep()

## Détails Revenus

In [24]:
display(display_multi_rev)
det_rev()

## Détails dépenses mensuels

In [25]:
display(display_multi_dep)
dep_mens()

C:\Users\clemm\AppData\Local\Temp\ipykernel_9248\3907845601.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Détails revenus mensuels

In [26]:
display(display_multi_rev)
rev_mens()

C:\Users\clemm\AppData\Local\Temp\ipykernel_9248\328975155.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Calcul nombre d'opération

## Catégories principales 

### Dépenses

##### Code

In [27]:
def calc_nb_ope_dep_ppal():
    df_cat_ppal_dep = sorted(df[(df["Catégorie_ppal"]!="Aucune") &
                            (~df["Débit"].isna())]["Catégorie_ppal"].unique())
    df_cat_ppal_dep = pd.DataFrame({"Catégorie_ppal":df_cat_ppal_dep, "Total":0})

    df_cat_ppal_dep_year = df[(df["Catégorie_ppal"]!="Aucune") &
                              (df["Date"].dt.year.isin(reconstruct_list(min(pick_year.value), max(pick_year.value)))) &
                              (~df["Débit"].isna())]
    df_cat_ppal_dep_year = df_cat_ppal_dep_year.groupby(by=["Catégorie_ppal"]).size().reset_index().sort_values(by=[0], ascending=False).rename({0:"Tot."}, axis=1)

    df_rename_cat_ppal = df_cat_ppal_dep.set_index('Catégorie_ppal').join(df_cat_ppal_dep_year.set_index('Catégorie_ppal'))
    df_rename_cat_ppal["Nombre"] = df_rename_cat_ppal["Total"] + df_rename_cat_ppal["Tot."]
    df_rename_cat_ppal = df_rename_cat_ppal.fillna(0).reset_index().sort_values(by=["Nombre", "Catégorie_ppal"], ascending=[False, True]).drop(["Total", "Tot."], axis=1)
    
    return df_rename_cat_ppal

##### Affichage

In [28]:
display(pick_year)
calc_nb_ope_dep_ppal()

IntRangeSlider(value=(2022, 2022), continuous_update=False, description='Pick a year :', max=2022, min=2019)

,Catégorie_ppal,Nombre
0,D01 - Courses,217.0
2,D03 - Restaurant,80.0
1,D02 - Achat en Ligne,73.0
3,D04 - Voiture,52.0
4,D05 - Loisir,46.0
5,D06 - Internet | Téléphone,25.0
10,D11 - Transport,19.0
8,D09 - Divers,18.0
6,D07 - Santé,17.0
9,D10 - Assurance,16.0


### Revenus

##### Code 

In [29]:
def calc_nb_ope_rev_ppal():
    df_cat_ppal_rev = sorted(df[(df["Catégorie_ppal"]!="Aucune") &
                            (~df["Crédit"].isna())]["Catégorie_ppal"].unique())
    df_cat_ppal_rev = pd.DataFrame({"Catégorie_ppal":df_cat_ppal_rev, "Total":0})

    df_cat_ppal_rev_year = df[(df["Catégorie_ppal"]!="Aucune") &
                              (df["Date"].dt.year.isin(reconstruct_list(min(pick_year.value), max(pick_year.value)))) &
                              (~df["Crédit"].isna())]
    df_cat_ppal_rev_year = df_cat_ppal_rev_year.groupby(by=["Catégorie_ppal"]).size().reset_index().sort_values(by=[0], ascending=False).rename({0:"Tot."}, axis=1)

    df_rename_cat_ppal = df_cat_ppal_rev.set_index('Catégorie_ppal').join(df_cat_ppal_rev_year.set_index('Catégorie_ppal'))
    df_rename_cat_ppal["Nombre"] = df_rename_cat_ppal["Total"] + df_rename_cat_ppal["Tot."]
    df_rename_cat_ppal = df_rename_cat_ppal.fillna(0).reset_index().sort_values(by=["Nombre", "Catégorie_ppal"], ascending=[False, True]).drop(["Total", "Tot."], axis=1)
    
    return df_rename_cat_ppal

##### Affichage

In [30]:
display(pick_year)
calc_nb_ope_rev_ppal()

IntRangeSlider(value=(2022, 2022), continuous_update=False, description='Pick a year :', max=2022, min=2019)

,Catégorie_ppal,Nombre
1,R02 - Remboursement,16.0
0,R01 - Salaire,15.0
3,R04 - Virement | Chèque,11.0
2,R03 - Santé,8.0
4,R05 - PEE,8.0
6,R07 - Aides,1.0
5,R06 - Banque,0.0
7,R08 - Logement | Caution,0.0


## Catégories secondaires

### Dépenses

In [31]:
df_cat_ppal_dep = sorted(df[(df["Catégorie_ppal"]!="Aucune") &
                        (~df["Débit"].isna())]["Catégorie_ppal"].unique())
df_cat_ppal_dep = pd.DataFrame({"Catégorie_ppal":df_cat_ppal_dep, "Total":0})

df_cat_ppal_dep_year = df[(df["Catégorie_ppal"]!="Aucune") &
                          (df["Date"].dt.year.isin(reconstruct_list(min(pick_year.value), max(pick_year.value)))) &
                          (~df["Débit"].isna())]
df_cat_ppal_dep_year = df_cat_ppal_dep_year.groupby(by=["Catégorie_ppal"]).size().reset_index().sort_values(by=[0], ascending=False).rename({0:"Tot."}, axis=1)

df_rename_cat_ppal = df_cat_ppal_dep.set_index('Catégorie_ppal').join(df_cat_ppal_dep_year.set_index('Catégorie_ppal'))
df_rename_cat_ppal["Nombre"] = df_rename_cat_ppal["Total"] + df_rename_cat_ppal["Tot."]
df_rename_cat_ppal = df_rename_cat_ppal.fillna(0).reset_index().sort_values(by=["Nombre", "Catégorie_ppal"], ascending=[False, True]).drop(["Total", "Tot."], axis=1)

df_rename_cat_ppal.head()

,Catégorie_ppal,Nombre
0,D01 - Courses,217.0
2,D03 - Restaurant,80.0
1,D02 - Achat en Ligne,73.0
3,D04 - Voiture,52.0
4,D05 - Loisir,46.0


In [32]:
df

,Date,Description,Catégorie,Mode de paiement,Tiers,Réf,Débit,Crédit,P,Catégorie_ppal,Catégorie_sec
0,2019-01-07,NaN,R04 - Virement | Chèque > 01 - Mireille Maisons,Non indiqué,Aucun,NaN,NaN,150.0,1.0,R04 - Virement | Chèque,01 - Mireille Maisons
1,2019-01-07,NaN,R04 - Virement | Chèque > xx - Carolane Dupont,Non indiqué,Aucun,NaN,NaN,325.0,1.0,R04 - Virement | Chèque,xx - Carolane Dupont
2,2019-01-07,NaN,D11 - Transport > 01 - Ilevia,Non indiqué,Aucun,NaN,14.45,NaN,1.0,D11 - Transport,01 - Ilevia
3,2019-01-07,NaN,D03 - Restaurant > 03 - KFC,Non indiqué,Aucun,NaN,29.00,NaN,1.0,D03 - Restaurant,03 - KFC
4,2019-01-07,NaN,R04 - Virement | Chèque > 04 - Gislaine Maisons,Non indiqué,Aucun,NaN,NaN,200.0,1.0,R04 - Virement | Chèque,04 - Gislaine Maisons
...,...,...,...,...,...,...,...,...,...,...,...
3371,2022-09-01,CCP > Livret,Aucune,Virement interne,Aucun,NaN,500.00,NaN,1.0,Aucune,None
3372,2022-09-01,CCP > Livret,Aucune,Virement interne,Aucun,NaN,NaN,500.0,1.0,Aucune,None
3373,2022-09-02,NaN,D01 - Courses > 05 - Leclerc,Non indiqué,Aucun,NaN,17.20,NaN,1.0,D01 - Courses,05 - Leclerc
3374,2022-09-02,NaN,D18 - Bricolage | Meuble | Déco > 11 - Conforama,Non indiqué,Aucun,NaN,139.99,NaN,1.0,D18 - Bricolage | Meuble | Déco,11 - Conforama


In [33]:
df.groupby(["Catégorie_ppal", "Catégorie_sec"]).size().sort_index(level=0)

Catégorie_ppal            Catégorie_sec       
D01 - Courses             00 - Divers              19
                          01 - Boulangerie        274
                          02 - Auchan             377
                          03 - Carrefour          125
                          04 - Intermarché         26
                                                 ... 
R07 - Aides               02 - Prime inflation      1
                          xx - Aide permis          1
                          xx - Bourse Mermoz        1
                          xx - CAF                  8
R08 - Logement | Caution  xx - Nadia Arian          1
Length: 267, dtype: int64

In [34]:
# df_cat_sec = sorted(df[~df["Catégorie_sec"].isna()]["Catégorie_sec"].unique())